## My Trainer

In [6]:
%cd /home/mkallel/explo/

from mushroom_rl.environments.dm_control_env import DMControl
from mushroom_rl.policy import DeterministicPolicy
from src.ddpg import DDPG
from src.helpers import setup_experiment
from src.config import get_configs
import torch
from src.approximators.actor import ActorNetwork
from src.approximators.critic import CriticNetwork


# MDP
horizon = 500
gamma = 0.99
gamma_eval = 1.
#mdp = DMControl('walker', 'stand', horizon, gamma)


# Settings
initial_replay_size = 500
max_replay_size = 10000
batch_size = 200
n_features = 80
tau = .001


# from os import path
# import logging
# #log_file_path = path.join()
# logging.config.fileConfig("/home/mkallel/explo/logging.conf")

/home/mkallel/explo


KeyError: 'formatters'

In [3]:
pwd

'/home/mkallel/explo'

In [2]:
pip install dm_control

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.2/38.2 MB 39.2 MB/s eta 0:00:0000:0100:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.6/142.6 kB 44.5 MB/s eta 0:00:00
  Using cached pyparsing-2.4.7-py2.py3-none-any.whl (67 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 66.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.8/4.8 MB 62.2 MB/s eta 0:00:0000:0100:01
  Using cached mujoco-2.2.1-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.7 MB)
  Created wheel for dm_control: filename=dm_control-1.0.4-py3-none-any.whl size=38550813 sha256=f6d6c5750f4c7302a2e3e735be5783d8ab740e0263c307939c478840ddd79927
  Stored in directory: /home/mkallel/.cache/pip/wheels/ec/ce/ae/797859ab9476bfa259701f141a2b46d4fad2b84ef7acb38284
Successfully built dm_control
  Attempting uninstall: pyparsing
    Found existing i

In [2]:
def fit_critic(agent,transitions,n_epochs=2):
    
    agent._replay_memory.add(transitions)

    state, action, reward, next_state, absorbing, _ =\
        agent._replay_memory.get(agent._batch_size())

    q_next = agent._next_q(next_state, absorbing)
    q_target = reward + agent.mdp_info.gamma * q_next

    agent._critic_approximator.fit(state, action, q_target,n_epochs=n_epochs,
                                    **agent._critic_fit_params)
    
    agent._update_target(agent._critic_approximator,
                        agent._target_critic_approximator)

In [3]:
%cd /home/q123/Desktop/explo/src

from src.optimizers.esq_pytorch import ESQOptimizer


env_name = "Swimmer-v4"
kernel_name = "rbf"


if __name__ == '__main__':
    
    ## Setup environment
    env_config,likelihood_config,kernel_config,optimizer_config,trainer_config = get_configs(env_name,kernel_name)
    env_config["manipulate_state"] = True
    
    policy_config = {
                "add_layer":[],### can be empty or [8,7] for adding 2 layers with width 8,7  neurons respectively
                "add_bias":False,
    }
    _,env = setup_experiment(env_config,kernel_config,likelihood_config,policy_config)


    # Setup DDPG
    mdp = env.env

    policy_class = DeterministicPolicy
    policy_params = dict()

    actor_input_shape = mdp.info.observation_space.shape
    actor_params = dict(network=ActorNetwork,
                        n_features=n_features,
                        input_shape=actor_input_shape,
                        output_shape=mdp.info.action_space.shape)

    actor_optimizer = {'class': torch.optim.Adam,
                    'params': {'lr': 1e-4}}

    critic_input_shape = (actor_input_shape[0] + mdp.info.action_space.shape[0],)
    
    critic_params = dict(network=CriticNetwork,
                        optimizer={'class': torch.optim.Adam,
                                    'params': {'lr': 1e-3}},
                        loss=F.mse_loss,
                        n_features=n_features,
                        input_shape=critic_input_shape,
                        output_shape=(1,))

    agent = DDPG(mdp.info, policy_class,policy_params,
                actor_params, actor_optimizer, 
                critic_params,
                batch_size, initial_replay_size, max_replay_size,
                tau)

    # esq_optimizer = ESQOptimizer(
    #                         critic=agent._critic_approximator.model.network,
    #                         actor = agent._actor_approximator.model.network,
    #                         sigma=1e-1,
    #                         params_per_step=40,
    #                         n_workers=8)
 
    # for i in range(100):
        
    #     avg_reward,states,transitions = env.run_many(agent._actor_approximator,5)
    #     print("avg_reward",avg_reward)  
            
    #     fit_critic(agent,transitions)
        
    #     states_batch,_,_,_,_,_ = agent._replay_memory.get(agent._batch_size())
        
    #     print("done training critic")
    #     grads = esq_optimizer.step(torch.tensor(states_batch)) ## fit critic
    #     #grads = esq_optimizer.step(states) ## fit critic
        
    #     #print(agent._critic_approximator.model.network.parameters())
        
    # #     #print(optimizer.actor.parameters())

/home/q123/Desktop/explo/src


ValueError: too many values to unpack (expected 5)

In [ ]:
mdp.info.action_space.shape[0]

2

In [ ]:
# from mushroom_rl.approximators import Regressor
# from mushroom_rl.approximators.parametric import TorchApproximator
# from mushroom_rl.utils.replay_memory import ReplayMemory

# replay_memory = ReplayMemory(initial_replay_size, max_replay_size)
# critic_approximator = Regressor(TorchApproximator,
#                                               **critic_params)
